In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Setup

### Configure your API keys

To fine-tune PaliGemma, you need to provide your HuggingFace Token and Roboflow API key. Follow these steps:

- Open your [`HuggingFace Settings`](https://huggingface.co/settings) page. Click `Access Tokens` then `New Token` to generate new token.
- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑).
    - Store HuggingFace Access Token under the name `HF_TOKEN`.
    - Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Select the runtime

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `L4 GPU`, and then click `Save`.

In [3]:
!nvidia-smi

Mon Sep 16 02:11:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN RTX               On  | 00000000:5E:00.0 Off |                  N/A |
|  0%   32C    P8              29W / 280W |      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Install dependencies

In [4]:
# !pip install -q roboflow maestro==0.2.0rc3
!pip install -q roboflow

## Download dataset

In [5]:
from roboflow import Roboflow
# from google.colab import userdata

# ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
ROBOFLOW_API_KEY = "O4znBVbtIB84A6sTVYh6"
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

version = rf.workspace("roboflow-jvuqo").project("poker-cards-fmjio").version(4)
dataset = version.download("florence2-od")

loading Roboflow workspace...
loading Roboflow project...


In [6]:
!head -n 5 {dataset.location}/train/annotations.jsonl

{"image":"IMG_20220316_172418_jpg.rf.e3cb4a86dc0247e71e3697aa3e9db923.jpg","prefix":"<OD>","suffix":"9 of clubs<loc_138><loc_100><loc_470><loc_448>10 of clubs<loc_388><loc_145><loc_670><loc_453>jack  of clubs<loc_566><loc_166><loc_823><loc_432>queen of clubs<loc_365><loc_465><loc_765><loc_999>king of clubs<loc_601><loc_440><loc_949><loc_873>"}
{"image":"IMG_20220316_171515_jpg.rf.e3b1932bb375b3b3912027647586daa8.jpg","prefix":"<OD>","suffix":"5 of clubs<loc_554><loc_2><loc_763><loc_467>6 of clubs<loc_399><loc_79><loc_555><loc_466>7 of clubs<loc_363><loc_484><loc_552><loc_905>8 of clubs<loc_535><loc_449><loc_757><loc_971>"}
{"image":"IMG_20220316_165139_jpg.rf.e30257ec169a2bfdfecb693211d37250.jpg","prefix":"<OD>","suffix":"9 of diamonds<loc_596><loc_535><loc_859><loc_982>jack of diamonds<loc_211><loc_546><loc_411><loc_880>queen of diamonds<loc_430><loc_34><loc_692><loc_518>king of diamonds<loc_223><loc_96><loc_451><loc_523>10 of diamonds<loc_387><loc_542><loc_604><loc_925>"}
{"image":"I

## Fine-tune Florence-2 on detection dataset

In [7]:
!maestro paligemma train --help

                                                                                
 Usage: maestro paligemma train [OPTIONS]                                       
                                                                                
 Train PaliGemma model                                                          
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ *  --dataset                                 TEXT     Path to the dataset    │
│                                                       used for training      │
│                                                       [default: None]        │
│                                                       [required]             │
│    --model_id                                TEXT     Identifier for the     │
│                                                       PaliGemma model        │
│                           

In [ ]:
!maestro paligemma train --dataset={dataset.location} --epochs=10 --batch_size=6 --lr=5e-6 --metrics mean_average_precision

Training configuration
{
    'dataset': 
'/mnt/nas2/users/sbchoi/multimodal-maestro/cookbooks/poker-cards-4',
    'model_id': 'google/paligemma-3b-pt-224',
    'revision': 'float16',
    'device': device(type='cuda', index=0),
    'cache_dir': None,
    'epochs': 10,
    'optimizer': 'adamw',
    'lr': 5e-06,
    'lr_scheduler': 'linear',
    'batch_size': 6,
    'val_batch_size': None,
    'num_workers': 0,
    'val_num_workers': None,
    'lora_r': 8,
    'lora_alpha': 8,
    'lora_dropout': 0.05,
    'bias': 'none',
    'use_rslora': True,
    'init_lora_weights': 'gaussian',
    'output_dir': './training/paligemma',
    'metrics': [
        <maestro.trainer.common.utils.metrics.MeanAveragePrecisionMetric object 
at 0x7f3348202290>
    ]
}
preprocessor_config.json: 100%|████████████████| 699/699 [00:00<00:00, 2.29MB/s]
tokenizer_config.json: 100%|████████████████| 40.0k/40.0k [00:00<00:00, 233kB/s]
tokenizer.model: 100%|█████████████████████| 4.26M/4.26M [00:00<00:00, 5.84MB/s]
toke

## Evaluate Florence-2 on object detection task

In [ ]:
!ls -la training/florence-2/1/metrics/

In [ ]:
from IPython.display import display, Image

Image(filename='training/florence-2/1/metrics/loss_plot.png', height=600)

In [ ]:
from IPython.display import display, Image

Image(filename='training/florence-2/1/metrics/map50:95_plot.png', height=600)

In [ ]:
!maestro florence2 evaluate \
--dataset={dataset.location} \
--model_id=/content/training/florence-2/1/checkpoints/best \
--metrics mean_average_precision

In [ ]:
!cat /content/evaluation/florence-2/metrics/evaluation.json

In [ ]:
!ls -la training/florence-2/1/checkpoints/

## Run inference

In [ ]:
from maestro.trainer.models.florence_2.checkpoints import load_model

processor, model = load_model(model_id_or_path="/content/training/florence-2/1/checkpoints/best")

In [ ]:
import supervision as sv
from maestro.trainer.common.data_loaders.datasets import JSONLDataset

ds = JSONLDataset(
    jsonl_file_path = f"{dataset.location}/valid/annotations.jsonl",
    image_directory_path = f"{dataset.location}/valid/"
)

image, _ = ds[0]
text = "<OD>"
task = "<OD>"

inputs = processor(text=text, images=image, return_tensors="pt").to("cuda")
generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
response = processor.post_process_generation(generated_text, task=task, image_size=image.size)
detections = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, response, resolution_wh=image.size)

box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)

image = box_annotator.annotate(image, detections)
image = label_annotator.annotate(image, detections)
image.thumbnail((600, 600))
image